In [61]:
import pandas as pd
import numpy as np
import warnings
from geopy.geocoders import Nominatim
import folium
import folium.plugins as pg


warnings.filterwarnings('ignore')

In [62]:
loc = './../NYC property sales Dataset/nyc-rolling-sales.csv'
df = pd.read_csv(loc)

In [63]:
df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,"153 AVENUE B, NEW YORK",...,5,0,5,1633,6440,1900,2,C2,6625000,19-07-2017 00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,"234 EAST 4TH STREET, NEW YORK",...,28,3,31,4616,18690,1900,2,C7,-,14-12-2016 00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,"197 EAST 3RD STREET, NEW YORK",...,16,1,17,2212,7803,1900,2,C7,-,09-12-2016 00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,"154 EAST 7TH STREET, NEW YORK",...,10,0,10,2272,6794,1913,2,C4,3936272,23-09-2016 00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,"301 EAST 10TH STREET, NEW YORK",...,6,0,6,2369,4615,1900,2,C2,8000000,17-11-2016 00:00


In [64]:
# Remove Column which are empty or not required
df.drop(columns=['Unnamed: 0','EASE-MENT'], axis=1, inplace=True)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   BOROUGH                         84548 non-null  int64 
 1   NEIGHBORHOOD                    84548 non-null  object
 2   BUILDING CLASS CATEGORY         84548 non-null  object
 3   TAX CLASS AT PRESENT            84548 non-null  object
 4   BLOCK                           84548 non-null  int64 
 5   LOT                             84548 non-null  int64 
 6   BUILDING CLASS AT PRESENT       84548 non-null  object
 7   ADDRESS                         84548 non-null  object
 8   APARTMENT NUMBER                84548 non-null  object
 9   ZIP CODE                        84548 non-null  int64 
 10  RESIDENTIAL UNITS               84548 non-null  int64 
 11  COMMERCIAL UNITS                84548 non-null  int64 
 12  TOTAL UNITS                     84548 non-null

In [66]:
np.transpose(df.describe())

,count,mean,std,min,25%,50%,75%,max
BOROUGH,84548.0,2.998758,1.289790,1.0,2.00,3.0,4.0,5.0
BLOCK,84548.0,4237.218976,3568.263407,1.0,1322.75,3311.0,6281.0,16322.0
LOT,84548.0,376.224015,658.136814,1.0,22.00,50.0,1001.0,9106.0
ZIP CODE,84548.0,10731.991614,1290.879147,0.0,10305.00,11209.0,11357.0,11694.0
RESIDENTIAL UNITS,84548.0,2.025264,16.721037,0.0,0.00,1.0,2.0,1844.0
COMMERCIAL UNITS,84548.0,0.193559,8.713183,0.0,0.00,0.0,0.0,2261.0
TOTAL UNITS,84548.0,2.249184,18.972584,0.0,1.00,1.0,2.0,2261.0
YEAR BUILT,84548.0,1789.322976,537.344993,0.0,1920.00,1940.0,1965.0,2017.0
TAX CLASS AT TIME OF SALE,84548.0,1.657485,0.819341,1.0,1.00,2.0,2.0,4.0


In [67]:
#Check for whitespaces or - in data and replace then with np.nan
counts_ = df.apply(lambda x: str(x).count("-"))
counts_space = df.apply(lambda x: str(x).count(" "))
print(f"There are {counts_.sum().sum()} '-' and {counts_space.sum().sum()} spaces")

df.replace(to_replace={' ':np.nan,'-':np.nan}, inplace=True)

There are 27 '-' and 2037 spaces


In [68]:
#EDA
def check_df(data, head=5):
    print("\n------Shape------")
    print(f'Shape     : {df.shape}\n'
          f'Size      : {df.size}\n'
          f'Dimension : {df.ndim}')
    print("\n------Types------")
    print(data.dtypes)
    print("\n------Head------")
    print(data.head(head))
    print("\n------Tail------")
    print(data.tail(head))
    print("\n------Missing Values------")
    print(data.isnull().sum())
    print("\n------Duplicated Values------")
    print(data.duplicated().sum())
    print("\n------Unique Values------")
    print(data.nunique())
    print("\n------Describe------")
    print(data.describe().T)

check_df(df)


------Shape------
Shape     : (84548, 20)
Size      : 1690960
Dimension : 2

------Types------
BOROUGH                            int64
NEIGHBORHOOD                      object
BUILDING CLASS CATEGORY           object
TAX CLASS AT PRESENT              object
BLOCK                              int64
LOT                                int64
BUILDING CLASS AT PRESENT         object
ADDRESS                           object
APARTMENT NUMBER                  object
ZIP CODE                           int64
RESIDENTIAL UNITS                  int64
COMMERCIAL UNITS                   int64
TOTAL UNITS                        int64
LAND SQUARE FEET                  object
GROSS SQUARE FEET                 object
YEAR BUILT                         int64
TAX CLASS AT TIME OF SALE          int64
BUILDING CLASS AT TIME OF SALE    object
SALE PRICE                        object
SALE DATE                         object
dtype: object

------Head------
   BOROUGH   NEIGHBORHOOD                      BUILD

In [69]:
#remove duplicates
print(f'Count of Duplicates:{df.duplicated().sum()}')
df.drop_duplicates(inplace=True)

Count of Duplicates:765


In [70]:
#Convert numerics float data to numeric format
numeric_columns = ['LAND SQUARE FEET', 'GROSS SQUARE FEET', 'SALE PRICE']
for col in df.columns:
    if col in numeric_columns:
        # Convert to numeric, setting errors to 'coerce' to handle non-numeric values
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [71]:
#Convert cat type column to object
df['TAX CLASS AT TIME OF SALE'] = df['TAX CLASS AT TIME OF SALE'].astype('object')

In [72]:
# Imputing missing values in 'LAND SQUARE FEET' and 'GROSS SQUARE FEET' using the median
for col in ['LAND SQUARE FEET', 'GROSS SQUARE FEET']:
    median_value = df[col].median()
    df[col].fillna(median_value, inplace=True)

# Removing rows where 'SALE PRICE' is missing
df = df.dropna(subset=['SALE PRICE'])

In [73]:
# Replacing borough with the names
df['BOROUGH'][df['BOROUGH'] == 1] = 'Manhattan'
df['BOROUGH'][df['BOROUGH'] == 2] = 'Bronx'
df['BOROUGH'][df['BOROUGH'] == 3] = 'Brooklyn'
df['BOROUGH'][df['BOROUGH'] == 4] = 'Queens'
df['BOROUGH'][df['BOROUGH'] == 5] = 'Staten Island'

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69607 entries, 0 to 84547
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         69607 non-null  object 
 1   NEIGHBORHOOD                    69607 non-null  object 
 2   BUILDING CLASS CATEGORY         69607 non-null  object 
 3   TAX CLASS AT PRESENT            68934 non-null  object 
 4   BLOCK                           69607 non-null  int64  
 5   LOT                             69607 non-null  int64  
 6   BUILDING CLASS AT PRESENT       68934 non-null  object 
 7   ADDRESS                         69607 non-null  object 
 8   APARTMENT NUMBER                15441 non-null  object 
 9   ZIP CODE                        69607 non-null  int64  
 10  RESIDENTIAL UNITS               69607 non-null  int64  
 11  COMMERCIAL UNITS                69607 non-null  int64  
 12  TOTAL UNITS                     69607

In [75]:
cat_cols = df.select_dtypes(exclude='number').columns.to_list()
cat_cols

['BOROUGH',
 'NEIGHBORHOOD',
 'BUILDING CLASS CATEGORY',
 'TAX CLASS AT PRESENT',
 'BUILDING CLASS AT PRESENT',
 'ADDRESS',
 'APARTMENT NUMBER',
 'TAX CLASS AT TIME OF SALE',
 'BUILDING CLASS AT TIME OF SALE',
 'SALE DATE']

In [76]:
for cols in cat_cols:
    print(cols)
    print(df[cols].unique())

BOROUGH
['Manhattan' 'Bronx' 'Brooklyn' 'Queens' 'Staten Island']
NEIGHBORHOOD
['ALPHABET CITY' 'CHELSEA' 'CHINATOWN' 'CIVIC CENTER' 'CLINTON'
 'EAST VILLAGE' 'FASHION' 'FINANCIAL' 'FLATIRON' 'GRAMERCY'
 'GREENWICH VILLAGE-CENTRAL' 'GREENWICH VILLAGE-WEST' 'HARLEM-CENTRAL'
 'HARLEM-EAST' 'HARLEM-UPPER' 'HARLEM-WEST' 'INWOOD' 'JAVITS CENTER'
 'KIPS BAY' 'LITTLE ITALY' 'LOWER EAST SIDE' 'MANHATTAN VALLEY'
 'MIDTOWN CBD' 'MIDTOWN EAST' 'MIDTOWN WEST' 'MORNINGSIDE HEIGHTS'
 'MURRAY HILL' 'ROOSEVELT ISLAND' 'SOHO' 'SOUTHBRIDGE' 'TRIBECA'
 'UPPER EAST SIDE (59-79)' 'UPPER EAST SIDE (79-96)'
 'UPPER EAST SIDE (96-110)' 'UPPER WEST SIDE (59-79)'
 'UPPER WEST SIDE (79-96)' 'UPPER WEST SIDE (96-116)'
 'WASHINGTON HEIGHTS LOWER' 'WASHINGTON HEIGHTS UPPER' 'BATHGATE'
 'BAYCHESTER' 'BEDFORD PARK/NORWOOD' 'BELMONT' 'BRONX PARK' 'BRONXDALE'
 'CASTLE HILL/UNIONPORT' 'CITY ISLAND' 'CITY ISLAND-PELHAM STRIP'
 'CO-OP CITY' 'COUNTRY CLUB' 'CROTONA PARK' 'EAST RIVER' 'EAST TREMONT'
 'FIELDSTON' 'FORDHAM' '

In [77]:
cat_cols.remove('ADDRESS')
cat_cols.remove('APARTMENT NUMBER')

In [83]:
df["ADDRESS SHORT"] = df['ADDRESS'].apply(lambda x: " ".join(x.split(' ')[1:]))

In [84]:
df_latlog = pd.DataFrame()
df_latlog['ZIP CODE'] = df['ADDRESS SHORT'].unique()

In [86]:
#Do a location based analysis
from time import sleep 

# Initialize the geolocator
geolocator = Nominatim(user_agent="name_of_the_agent", timeout=10)

def lat_long_from_zip(x):
    # Add a random sleep interval to avoid overloading the geocoder
    sleep(np.random.uniform(0.1, 0.2))
    
    try:
        location = geolocator.geocode(x, addressdetails=True)
        if location:
            return pd.Series({'LATITUDE': location.latitude, 'LONGITUDE': location.longitude})
        else:
            return pd.Series({'LATITUDE': np.nan, 'LONGITUDE': np.nan})
    except Exception as e:
        print(f"Error geocoding {x}: {e}")
        return pd.Series({'LATITUDE': np.nan, 'LONGITUDE': np.nan})


# Apply the function to each zip code and assign to new columns
df_latlog[['LATITUDE', 'LONGITUDE']] = df_latlog['ZIP CODE'].apply(lambda x: lat_long_from_zip(x))


In [22]:
df = pd.merge(df, df_latlog, on='ZIP CODE',how='left')

In [23]:
# PLot the heatmap of house prices using foliumn
center_lat = df[df['LATITUDE'].notna()]['LATITUDE'].mean()
center_lon = df[df['LONGITUDE'].notna()]['LATITUDE'].mean()

MAP = folium.Map(location=[center_lat,center_lon],zoom_start=10)

In [28]:
heat_data = [[row['LATITUDE'], row['LONGITUDE'], row['SALE PRICE']] for index, row in df[df['LATITUDE'].notna()].iterrows()]
pg.HeatMap(heat_data).add_to(MAP)

In [30]:
MAP